In [0]:
#!import "Storage"

In [0]:
#!import "../Utils/ExtentionMethods"

# Basic Cashflow

In [0]:
public interface ComputeBasicCashflow : IScope<BasicCashflow>
{
    static ApplicabilityBuilder ScopeApplicabilityBuilder(ApplicabilityBuilder builder) =>
     builder.ForScope<ComputeBasicCashflow>(s => s
         .WithApplicability<ConstantShape>(x => x.Identity.Shape == Shape.Constant)
         .WithApplicability<LinearShape>(x => x.Identity.Shape == Shape.Linear)
         .WithApplicability<CompoundShape>(x => x.Identity.Shape == Shape.Compound)
    );
    //Custom is the default Case
    double[] Values => Identity.Values;
}

public interface ConstantShape : ComputeBasicCashflow {
    double[] ComputeBasicCashflow.Values => Identity.InitialValue.GetConstantValues(Identity.Shift, Identity.Length);
}

public interface LinearShape : ComputeBasicCashflow {
    double[] ComputeBasicCashflow.Values => Identity.InitialValue.GetLinearValues(Identity.Shift, Identity.Length, Identity.Values);
}

public interface CompoundShape : ComputeBasicCashflow {
    double[] ComputeBasicCashflow.Values => Identity.InitialValue.GetCompoundValues(Identity.Shift, Identity.Length, Identity.Values);

}

In [0]:
public interface BasicCashflows : IScope<CashflowIdentity, CashflowStorage>
{
    public IEnumerable<(string Layer, double[] Values)> LayerValues => GetStorage().GetBasicCashflows(Identity)
        .Select((cd,i) => ("Basic" + i.ToString(), GetScope<ComputeBasicCashflow>(cd).Values));
}

# Bounded Cashflow

In [0]:
public interface ComputeBoundedCashflows : IScope<BoundedCashflow>
{
    static ApplicabilityBuilder ScopeApplicabilityBuilder(ApplicabilityBuilder builder) =>
     builder.ForScope<ComputeBoundedCashflows>(s => s
         .WithApplicability<RandomShape>(x => x.Identity.Shape == Shape.Random)
    );
    //null is the default Case
    double[] Values => default;
}

public interface RandomShape: ComputeBoundedCashflows{
    double[] ComputeBoundedCashflows.Values => GetBoundedRandomValues(Identity.Shift, Identity.Length, Identity.Maximum, Identity.Minimum);
}

In [0]:
public interface BoundedCashflows : IScope<CashflowIdentity, CashflowStorage>
{
    public IEnumerable<(string Layer, double[] Values)> LayerValues => GetStorage().GetBoundedCashflows(Identity)
        .Select((cd,i) => ("Bounded" + i.ToString(), GetScope<ComputeBoundedCashflows>(cd).Values));     
}

# Reference Cashflow

In [0]:
public interface ReferencedCashflows : IScope<(CashflowIdentity Id, string OriginalAmountType), CashflowStorage>{
   static ApplicabilityBuilder ScopeApplicabilityBuilder(ApplicabilityBuilder builder) =>
        builder.ForScope<ReferencedCashflows>(s => s
            .WithApplicability<ParametrisedCashflows>(x => x.Identity.OriginalAmountType == x.Identity.Id.AmountType));
        
    public double[] Values => GetScope<ReferenceCashflows>(Identity.Id).LayerValues.Concat(
        GetScope<ParametrisedCashflows>(Identity).LayerValues)
        .Select(x => x.Values).AggregateDoubleArray();
}

public interface ParametrisedCashflows : ReferencedCashflows{
    public IEnumerable<(string Layer, double[] Values)> LayerValues => GetScope<BasicCashflows>(Identity.Id).LayerValues.Concat(
        GetScope<BoundedCashflows>(Identity.Id).LayerValues);
    double[] ReferencedCashflows.Values => LayerValues.Select(x => x.Values).AggregateDoubleArray();
}

public interface WeightedReference : IScope<(CashflowIdentity Id, string ReferenceAmountType), CashflowStorage>{
       static ApplicabilityBuilder ScopeApplicabilityBuilder(ApplicabilityBuilder builder) =>
        builder.ForScope<WeightedReference>(s => s
            .WithApplicability<WeightedReferenceSameAt>(x => x.Identity.ReferenceAmountType == x.Identity.Id.AmountType));

    //private
    double[] referencedValues => GetScope<ReferencedCashflows>((Identity.Id with {AmountType = Identity.ReferenceAmountType}, Identity.Id.AmountType)).Values;
    double[] weights => GetStorage().GetReferencedCashflow(Identity.Id).Single(cd => cd.ReferenceAmountType == Identity.ReferenceAmountType).Values;
    
    double[] Values => weights.Zip(referencedValues, (weight, referencedValue) => weight * referencedValue).ToArray();
}

public interface WeightedReferenceSameAt : WeightedReference{
    double[] WeightedReference.Values => weights.Zip(referencedValues, (weight, referencedValue) => ( weight - 1d )  * referencedValue).ToArray();
}

public interface ReferenceCashflows : IScope<CashflowIdentity, CashflowStorage>
{
    public IEnumerable<(string Layer, double[] Values)> LayerValues => GetStorage().GetReferencedCashflow(Identity)
        .Select((cd,i) => ("Reference" + i.ToString(), GetScope<WeightedReference>((Identity, cd.ReferenceAmountType)).Values));
}

# Compute all Cashflows

In [0]:
public interface Compute : IScope<CashflowIdentity, CashflowStorage>{
    public FpaVariable[] FpaVariables => GetScope<BasicCashflows>(Identity).LayerValues.Concat(
        GetScope<BoundedCashflows>(Identity).LayerValues).Concat(
        GetScope<ReferenceCashflows>(Identity).LayerValues).Select(v => 
        new FpaVariable() {
            Layer = v.Layer,
            Values = v.Values,
            DataNode = Identity.DataNode,
            AmountType = Identity.AmountType,
            AccidentYear = Identity.AccidentYear,
            EstimateType = EstimateTypes.BE,
            Novelty = Novelties.C,
            AocType = AocTypes.CL
        }).ToArray();
}